# Multi-Sensor Atmospheric Correction in Google Earth Engine by Image

**Description:** This script allows to do atmospheric correction on only individual images of Sentinel-2 and Landsat sensors, especifically for images over coastal or oceanic areas. Some atmospheric correction settings can be modified in the *parameters.py* module to work with images over inland areas (See line 36 in that module). The script does AC automatically by providing the right satellite mission (**mission**), list of images (**imageID**), and a specific GEE Asset (**assetID**) to export processed images to your personal GEE account.<br/>
More sensors can be added by modifying the *mission_specifics.py* and *parameters.py* modules to properly work with the available collections in GEE and [Py6S](https://github.com/robintw/Py6S/blob/master/Py6S/Params/wavelength.py).<br/>

Script modified from https://github.com/samsammurphy/gee-atmcorr-S2<br/>
By Luis Lizcano-Sandoval<br/>
College of Marine Science, University of South Florida<br/>
luislizcanos@usf.edu<br/>
Created: 10/30/2020<br/>
Updated: 09/02/2021

### Import modules and initialize Earth Engine



In [1]:
import ee
from Py6S import *
import datetime
import math
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
import mission_specifics as mn
import getBOA
import timeit

ee.Initialize()

### Earth Engine Collections
Set the collection of interest using one this specific categories:


In [10]:
mission = 'Sentinel2'
#mission = 'Landsat8'
#mission = 'Landsat7'
# mission = 'Landsat5'

## Define user asset and folder to save output in GEE:
userAsset = 'users/lizcanosandoval/BOA/'
outputFolder = 'FL_19'

### Image ID
Paste the ID of your target image.

In [11]:
imageID = '20190117T161619_20190117T162429_T17RLM' #Sentinel-2
#imageID = 'LC08_017040_20191130' #Landsat8
#imageID = 'LE07_015043_20001018' #Landsat7
# imageID = 'LT05_016041_19890205' #Landsat5

### Load image
Get the respective scene from the collection

In [12]:
# Load image
image = ee.Image(mn.eeCollection(mission) +'/'+ imageID)
print('Image: ', image.getInfo()['properties']['system:index'])

# Date
dateString = datetime.datetime.utcfromtimestamp(image.get('system:time_start').getInfo()/1000).strftime("%Y-%m-%d")
print('Date: ',dateString)

print('Mission: ', mission)

Image:  20190117T161619_20190117T162429_T17RLM
Date:  2019-01-17
Mission:  Sentinel2


### Atmospheric Correction
Available bands available for atmospheric correction for each sensor, according to the [Py6S module](https://github.com/robintw/Py6S/blob/master/Py6S/Params/wavelength.py): <br/>
* **Sentinel2:** ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12'] {Can Skip B6,B7,B8A,B9,B10},<br/>
* **Landsat8:** ['B1','B2','B3','B4','B5','B6','B7','B8','B9'] {B10,B11 - Thermal}{Can Skip B8,B9,B11},<br/>
* **Landsat7:** ['B1','B2','B3','B4','B5','B7'] {B6_VCID_1 - Thermal},<br/>
* **Landsat5:** ['B1','B2','B3','B4','B5','B7'] {B6 - Thermal},<br/>
* **Landsat4:** ['B1','B2','B3','B4','B5','B7'] {B6 - Thermal}<br/>

The respective QA band will be preserved after the correction:<br/>
* **Sentinel2:** 'QA60',<br/>
* **Landsat sensors:** 'BQA'

In [13]:
## Default bands of interest:
if 'Sentinel' in mission:
    bands = ['B1','B2','B3','B4','B5','B8','B11','B12'] #Sentinel-2
elif 'Landsat8' in mission:
    bands = ['B1','B2','B3','B4','B5','B6','B7'] #Landsat-8
else:
    bands = ['B1','B2','B3','B4','B5','B7'] #Landsat-7/5

**NOTE:** Sentinel's B8 might show negative reflectances on coastal and oceanic areas, but not on cloudy pixels, so it does not affect cloud masking procedures. I only use bands B8, B11, B12 for cloud masking. If you need to use these bands for other purposes just check the areas of negative values are not large, otherwise I would not recommend to use them (it is up to you). The negative reflectances might be due to overestimations of aerosols at sea-level in coastal areas and suspendend particles in water. Band B10 should not be provided as a surface reflectance output, because it does not provide information on the surface but on the cirrus clouds [[Main-Knorn et al. 2017]](https://www.researchgate.net/publication/320231869_Sen2Cor_for_Sentinel-2). Landsat sensors may show a similar behaviour in infrared bands. 

Run atmospheric correction (for Sentinel, it takes ~20s per image (8 bands)).

In [6]:
%%time
boaImage = getBOA.forImage(image, mission, bands)

## Verify that each band is present in the output:
print('Output bands: ', boaImage.bandNames().getInfo())

Done!
Output bands:  ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B6', 'BQA']
Wall time: 15.1 s


Rename the variable 'mission' if working with Sentinel-2

In [7]:
# If working with S-2, then identify whether the sensor is Sentinel-2A or Sentinel-2B, and rename the variable 'mission'
if 'Sentinel2' == mission:
    mission = str(image.getInfo()['properties']['SPACECRAFT_NAME'])

### Display results
Shows the image as example.

In [9]:
from IPython.display import display, Image

region = image.geometry().buffer(5000).bounds().getInfo()['coordinates']

# RGB Bands
channels = []
if 'Sentinel' in mission or 'Landsat8' == mission:
    channels = ['B4','B3','B2'] #For Sentinel & Landsat8
else:
    channels = ['B3','B2','B1'] #For Landsat7-5-4

# Display images:
original = Image(url=mn.TOA(image,mission).select(channels).getThumbUrl({
    'dimensions': '1000x1000',
    'min':0,
    'max':0.25
    }))

corrected = Image(url=boaImage.select(channels).getThumbUrl({
    'dimensions': '1000x1000',
    'min':0,
    'max':0.25,
    'gamma':1.8
    }))

display(original, corrected)

### Export to Asset

NOTE: 
* Be aware that each band will be resampled at the scale used to export the image. This will impact the size of the exported file.
* A Sentinel-2 image with 8 bands at 10m res each can occupy ~1.5 gb.
* A Sentinel-2 image with 8 bands can take ~4-8 min to ingest.

In [ ]:
# Some settings before exporting:
scale = []
sat = []
tile = []
if 'Sentinel' in mission:
    sat = 'Sentinel'
    scale = 10 #For Sentinel
    tile = image.getInfo()['properties']['MGRS_TILE']
else:
    sat = 'Landsat'
    scale = 30 #For Landsat 
    tile = str(image.getInfo()['properties']['WRS_PATH'])+str(image.getInfo()['properties']['WRS_ROW'])
    
# set some properties for export
output = output.set({'satellite': mission,
               'tile_id': str(tile),
               'file_id': imageID,                                               
               'date': dateString,
               'generator': 'Lizcano-Sandoval',
                    })

# define YOUR assetID. (This do not create folders, you need to create them manually)
assetID = 'users/lizcanosandoval/BOA/'+sat+'/'+outputFolder+'/' ##This goes to an ImageCollection folder
fileName = tile+'_'+imageID+'_BOA'
path = assetID + fileName

In [ ]:
## export
print('Submitting...')
export = ee.batch.Export.image.toAsset(\
        image = boaImage,                                                    
        description = 'BOA_'+imageID,
        assetId = path,
        region = image.geometry().buffer(10),                                      
        maxPixels = 1e9,
        scale = scale)

export.start()
print('Submitted!')